In [72]:
import pandas as pd 
import numpy as np
# TODO try sneaky 
train = pd.read_csv("raw/train.csv") # 136*43 matrix
test = pd.read_csv("raw/test.csv") #99999*42 matrix
print train.shape, test.shape

(137, 43) (100000, 42)


In [73]:
# convert date to days
import time
from datetime import datetime as dt
# train
all_diff = []
for item in train["Open Date"]:
    diff = dt.now() - dt.strptime(item, "%m/%d/%Y")
    all_diff.append(int(diff.days/365))

train['DaysOfRest'] = pd.Series(all_diff)

# test
all_diff = []
for item in test["Open Date"]:
    diff = dt.now() - dt.strptime(item, "%m/%d/%Y")
    all_diff.append(int(diff.days/365))

test['DaysOfRest'] = pd.Series(all_diff)
print train.shape, test.shape

(137, 44) (100000, 43)


In [74]:
# drop insufficient columns
train = train.drop('Id', 1)
train = train.drop('Open Date', 1)
test = test.drop('Id', 1)
test = test.drop('Open Date', 1)
print train.keys()
print test.keys()

Index([u'City', u'City Group', u'Type', u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'revenue', u'DaysOfRest'], dtype='object')
Index([u'City', u'City Group', u'Type', u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'DaysOfRest'], dtype='object')


In [75]:
# normalize arguments
train_normal = train.copy()

# preprocess train
names = [name for name in train.keys() if name[0]=='P']
max_params = []
j = 0
for i in range(0,len(train_normal)):
    max_value = 0
    for name in names:
        if train_normal.iloc[i][name] > max_value:
            max_value = train_normal.iloc[i][name]
    if max_value > 5:
        j +=1
    #print max_value
    max_params.append(max_value)
    
train_normal['max_params'] = pd.Series(max_params)

print train.keys()
print train.shape

Index([u'City', u'City Group', u'Type', u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'revenue', u'DaysOfRest'], dtype='object')
(137, 42)


In [76]:
# normalize train arguments
multiplier = {'P1':3,'P2':1.5,'P3':1.5,'P4':1.5,'P5':2,'P6':2,'P7':2,'P8':2,'P9':2,'P10':2,'P11':2,'P12':2,'P13':1.5,'P14':3,
              'P15':2,'P16':3,'P17':3,'P18':3,'P19':5,'P20':3,'P21':3,'P22':1,'P23':5,'P24':2,'P25':2,'P26':2.5,'P27':2.5,
              'P28':2.5,'P29':2.5,'P30':5,'P31':3,'P32':5,'P33':2,'P34':6,'P35':3,'P36':4,'P37':2}

for i in range(0,len(train_normal)):
    if train_normal.iloc[i]['max_params'] > 5:
        for name in names:
            train_normal[name]=train_normal[name].astype(np.float64)
            train_normal.loc[i,name] /= multiplier[name]

In [6]:
# normalize test arguments
test_normal = test.copy()
for i in range(0,len(test)):
    for name in names:
        val = test_normal.iloc[i][name]
        if val % 1 != 0 or val > 5:
            test_normal.loc[i,name] /= multiplier[name]
        #if name == 'P7' and val == 2: # probably useless
        #    test_normal.loc[i,name] /= 2
        #if name == 'P29' and val == 5:
        #    test_normal.loc[i,name] /= 2.5
    if i % 5000 == 0:
        print i
print "End"

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
End


In [77]:
# replace low freq city
big_city = ['İstanbul', 'Ankara', 'İzmir']
for i in range(0,len(train_normal)):
    if train_normal.iloc[i]["City"] not in big_city:
        train_normal.loc[i,"City"] = 'Other'

for i in range(0,len(test_normal)):
    if test_normal.iloc[i]["City"] not in big_city:
        test_normal.loc[i,"City"] = 'Other'    

In [68]:
# drop city type
meaningless_features = ['City Group','P3','P7','P9','P25','P27','P30','P31','P35','P37']
train_normal = train_normal.drop(meaningless_features,1)
test_normal = test_normal.drop(meaningless_features,1)


In [81]:
# remove outlier from train set and max_params
outlier_index = [16]
train_normal = train_normal.drop('max_params',1)
train_normal = train_normal.drop(train_normal.index[outlier_index])
train_normal = train_normal.reset_index(drop=True)
print len(train_normal)


136


In [125]:
# conversion to categorical variables
train_categorial = train_normal.copy()
test_categorial = test_normal.copy()
names = [name for name in test_categorial.keys() if name[0]=='P']
for item in names:
    train_categorial[item] = train_categorial[item].astype(np.int64)
    #train_categorial[item] = train_categorial[item].astype('category')
    test_categorial[item] = test_categorial[item].astype(np.int64)
    #test_categorial[item] = test_categorial[item].astype('category')

# select all null
null_categorical = train_categorial[train_categorial["P14"] == 0]
null_categorical = pd.get_dummies(null_categorical)
null_categorical = null_categorical.drop('revenue', 1)
null_categorical = null_categorical.reset_index(drop=True)
null_revenue = train_categorial[train_categorial["P14"] == 0].revenue 
null_revenue = null_revenue.reset_index(drop=True)

# make all not null forced
null_names = [name for name in null_categorical.keys() if null_categorical.iloc[0][name] == 0 and name[0]=='P']
forced_null = train_categorial.copy()
print null_names
for name in null_names:
    forced_null[name] = 0
forced_null = forced_null.drop('revenue', 1)
forced_null = pd.get_dummies(forced_null)
print train_categorial.keys()
print train_categorial.shape

['P14', 'P15', 'P16', 'P17', 'P18', 'P24', 'P25', 'P26', 'P27', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37']
Index([u'City', u'City Group', u'Type', u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'revenue', u'DaysOfRest', u'max_params'], dtype='object')
(136, 43)


In [126]:
# one-hot encoding
all_categorial_train = pd.get_dummies(train_categorial)
all_categorial_train["DaysOfRest"] = train_categorial["DaysOfRest"]
all_categorial_train = all_categorial_train.drop('revenue', 1)
all_categorial_train = all_categorial_train.drop('max_params', 1)

all_cat_test = pd.get_dummies(test_categorial)
all_cat_test["DaysOfRest"] = test_categorial["DaysOfRest"]
print all_categorial_train.keys()
print all_categorial_train.shape

Index([u'P1', u'P2', u'P3', u'P4', u'P5', u'P6', u'P7', u'P8', u'P9', u'P10', u'P11', u'P12', u'P13', u'P14', u'P15', u'P16', u'P17', u'P18', u'P19', u'P20', u'P21', u'P22', u'P23', u'P24', u'P25', u'P26', u'P27', u'P28', u'P29', u'P30', u'P31', u'P32', u'P33', u'P34', u'P35', u'P36', u'P37', u'DaysOfRest', u'City_Ankara', u'City_Other', u'City_İstanbul', u'City_İzmir', u'City Group_Big Cities', u'City Group_Other', u'Type_DT', u'Type_FC', u'Type_IL'], dtype='object')
(136, 47)


In [127]:
# print diff in keys and delete them
key_diff = set(all_cat_test.keys()) - set(all_categorial_train.keys())
print key_diff
for name in key_diff:
    all_cat_test = all_cat_test.drop(name, 1)
print set(all_cat_test.keys()) - set(all_categorial_train.keys())
print set(all_categorial_train.keys()) - set(all_cat_test.keys()) 
    

set(['Type_MB'])
set([])
set([])


In [128]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
train_two = pd.DataFrame(poly.fit_transform(all_categorial_train))
test_two = pd.DataFrame(poly.fit_transform(all_cat_test))
print train_two.shape
print test_two.shape

(136, 1176)
(100000, 1176)


In [130]:
# build model
# all in one
from sklearn import cross_validation, linear_model,ensemble
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.svm import SVR

dataset = train_two.copy()
Y = np.log(train_normal.revenue)

kf = cross_validation.KFold(len(dataset), n_folds=10,shuffle=True)
print kf

# simple regression
regr = linear_model.LinearRegression()
avg_rmse = []

print "============"
print "Simple regression"
print "============"

for train_index, test_index in kf:
    regr.fit(dataset.iloc[train_index], Y[train_index])
    pred = np.exp(regr.predict(dataset.iloc[test_index]))
    rmse = np.sqrt(np.mean((pred - np.exp(Y[test_index])) ** 2))
    avg_rmse.append(rmse)
    print("RMSE: %.2f" % rmse)

regr.fit(dataset,Y)

print("AVG RMSE: %.2f MAX RMSE: %.2f MIN RMSE: %.2f" % (np.mean(avg_rmse), np.max(avg_rmse), np.min(avg_rmse)))

print "============"
print "Elastic net"
print "============"

ratio = [.1, .5, .7, .9,.95, .99, 1] #.20,.35,.5,.75,
elastic = linear_model.ElasticNetCV(l1_ratio=1, eps=0.0001, n_alphas=100, fit_intercept=True, normalize=True, precompute='auto', max_iter=1000, tol=0.0001, cv=8, copy_X=True)
avg_rmse = []

for train_index, test_index in kf:
    elastic.fit(dataset.iloc[train_index], Y[train_index])
    pred = np.exp(elastic.predict(dataset.iloc[test_index]))
    rmse = np.sqrt(np.mean((pred - np.exp(Y[test_index])) ** 2))
    avg_rmse.append(rmse)
    print("RMSE: %.2f" % rmse)
    
elastic.fit(dataset,Y)
#dataset["elastic"] = pd.Series(elastic.predict(dataset))
print("AVG RMSE: %.2f MAX RMSE: %.2f MIN RMSE: %.2f" % (np.mean(avg_rmse), np.max(avg_rmse), np.min(avg_rmse)))

print "============"
print "GBM"
print "============"

params = {'n_estimators': 2000, 'max_depth': 6, 'min_samples_split': 2,
          'learning_rate': 0.001, 'loss': 'ls', 'min_samples_leaf':3}
clf = ensemble.GradientBoostingRegressor(**params)
avg_rmse = []

for train_index, test_index in kf:
    clf.fit(dataset.iloc[train_index], Y[train_index])
    pred = np.exp(clf.predict(dataset.iloc[test_index]))
    rmse = np.sqrt(np.mean((pred - np.exp(Y[test_index])) ** 2))
    avg_rmse.append(rmse)
    print("RMSE: %.2f" % rmse)

clf.fit(dataset,Y)
print("AVG RMSE: %.2f MAX RMSE: %.2f MIN RMSE: %.2f" % (np.mean(avg_rmse), np.max(avg_rmse), np.min(avg_rmse)))

print "============"
print "AdaBoost"
print "============"


ada_tr = ensemble.AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=10, loss='square')
avg_rmse = []

for train_index, test_index in kf:
    ada_tr.fit(dataset.iloc[train_index], Y[train_index])
    pred = np.exp(ada_tr.predict(dataset.iloc[test_index]))
    rmse = np.sqrt(np.mean((pred - np.exp(Y[test_index])) ** 2))
    avg_rmse.append(rmse)
    print("RMSE: %.2f" % rmse)

ada_tr.fit(dataset,Y)
print("AVG RMSE: %.2f MAX RMSE: %.2f MIN RMSE: %.2f" % (np.mean(avg_rmse), np.max(avg_rmse), np.min(avg_rmse)))

print "============"
print "Extra Trees"
print "============"


extr = ExtraTreeRegressor(criterion='mse', max_depth=6, min_samples_split=3, min_samples_leaf=3, max_features='auto')
avg_rmse = []

for train_index, test_index in kf:
    extr.fit(dataset.iloc[train_index], Y[train_index])
    pred = np.exp(extr.predict(dataset.iloc[test_index]))
    rmse = np.sqrt(np.mean((pred - np.exp(Y[test_index])) ** 2))
    avg_rmse.append(rmse)
    print("RMSE: %.2f" % rmse)

extr.fit(dataset,Y)
print("AVG RMSE: %.2f MAX RMSE: %.2f MIN RMSE: %.2f" % (np.mean(avg_rmse), np.max(avg_rmse), np.min(avg_rmse)))


print "============"
print "SGD"
print "============"

sgd = SGDRegressor(penalty='elasticnet', alpha=0.01, l1_ratio=0.25, fit_intercept=True)
avg_rmse = []

for train_index, test_index in kf:
    sgd.fit(dataset.iloc[train_index], Y[train_index])
    pred = np.exp(sgd.predict(dataset.iloc[test_index]))
    rmse = np.sqrt(np.mean((pred - np.exp(Y[test_index])) ** 2))
    avg_rmse.append(rmse)
    print("RMSE: %.2f" % rmse)

sgd.fit(dataset,Y)
print("AVG RMSE: %.2f MAX RMSE: %.2f MIN RMSE: %.2f" % (np.mean(avg_rmse), np.max(avg_rmse), np.min(avg_rmse)))

print "============"
print "Random Forest"
print "============"
rfr = ensemble.RandomForestRegressor(n_estimators=200, min_samples_leaf = 3, oob_score=True,  max_features=None)
avg_rmse = []

for train_index, test_index in kf:
    rfr.fit(dataset.iloc[train_index], Y[train_index])
    pred = np.exp(rfr.predict(dataset.iloc[test_index]))
    rmse = np.sqrt(np.mean((pred - np.exp(Y[test_index])) ** 2))
    avg_rmse.append(rmse)
    print("RMSE: %.2f" % rmse)

rfr.fit(dataset,Y)
print("AVG RMSE: %.2f MAX RMSE: %.2f MIN RMSE: %.2f" % (np.mean(avg_rmse), np.max(avg_rmse), np.min(avg_rmse)))

print "============"
print "SVR"
print "============"
svr = SVR(C=1, epsilon=0.1)
avg_rmse = []
for train_index, test_index in kf:
    svr.fit(dataset.iloc[train_index], Y[train_index])
    pred = np.exp(svr.predict(dataset.iloc[test_index]))
    rmse = np.sqrt(np.mean((pred - np.exp(Y[test_index])) ** 2))
    avg_rmse.append(rmse)
    print("RMSE: %.2f" % rmse)
    
svr.fit(dataset,Y)
print("AVG RMSE: %.2f MAX RMSE: %.2f MIN RMSE: %.2f" % (np.mean(avg_rmse), np.max(avg_rmse), np.min(avg_rmse)))


sklearn.cross_validation.KFold(n=136, n_folds=10, shuffle=True, random_state=None)
Simple regression
RMSE: 3444332.56
RMSE: 3582273.91
RMSE: 2645345.81
RMSE: 70454370.60
RMSE: 3255044.55
RMSE: 3790895.90
RMSE: 2295451.69
RMSE: 32751588.37
RMSE: 3101765.85
RMSE: 4068278.59
AVG RMSE: 12938934.78 MAX RMSE: 70454370.60 MIN RMSE: 2295451.69
Elastic net
RMSE: 1408713.52
RMSE: 2037927.76
RMSE: 1554568.21
RMSE: 1527577.15
RMSE: 1730022.45
RMSE: 2309793.64
RMSE: 1252813.05
RMSE: 3808869.24
RMSE: 2480923.08
RMSE: 1853317.42
AVG RMSE: 1996452.55 MAX RMSE: 3808869.24 MIN RMSE: 1252813.05
GBM
RMSE: 1328220.62
RMSE: 2031495.81
RMSE: 1620515.78
RMSE: 1486181.59
RMSE: 1648296.76
RMSE: 2203065.39
RMSE: 1372275.08
RMSE: 3761522.22
RMSE: 2631627.12
RMSE: 2019018.97
AVG RMSE: 2010221.94 MAX RMSE: 3761522.22 MIN RMSE: 1328220.62
AdaBoost
RMSE: 1375351.76
RMSE: 2291729.78
RMSE: 1460746.58
RMSE: 1222059.19
RMSE: 1892626.44
RMSE: 3309125.54
RMSE: 1583621.40
RMSE: 4020152.49
RMSE: 2791333.31
RMSE: 1566135.39
A

In [101]:
for i in range(0, len(clf.feature_importances_)):
    if clf.feature_importances_[i]*100 > 1:
        print i, clf.feature_importances_[i]*100
    

0 15.4216888794
2 1.47555748486
4 4.97145929936
5 1.53329790066
14 2.92573333672
15 5.87624666696
37 1.13712895742
46 3.08716245893


KeyboardInterrupt: 

In [113]:
dataset = all_cat_test.copy()
#dataset["regr"] = pd.Series(regr.predict(dataset))
#prediction = dataset["regr"] # regr.predict(dataset)
prediction1 = rfr.predict(dataset)

ValueError: Number of features of the model must  match the input. Model n_features is 196 and  input n_features is 48 

In [25]:
prediction = np.exp(prediction1)
#prediction = prediction1
prediction1 = np.exp(prediction1)

In [26]:
%matplotlib
print np.max(prediction1)
print np.min(prediction1)
#print pd.Series(prediction).hist(bins=100)
print len(prediction1)
ids = []
ids2 = []
for i in range(0,len(prediction)):
    if prediction[i] > 20000000:
        ids.append(i)
    if prediction1[i] > 10000000:
        ids2.append(i)
print len(ids), len(ids2)
slice_of_solution = test_normal.iloc[ids]
#for name in slice_of_solution.keys():
#    print name
#    print dict(train_normal[name].value_counts())
#    print dict(slice_of_solution[name].value_counts())

Using matplotlib backend: MacOSX
7938037.41983
1998296.63421
100000
0 0


In [27]:
import csv
name = "./raw/my_submission_4_gbm_poly.csv"
result = open(name, "w")
writer = csv.writer(result, delimiter=',')
writer.writerow(["Id","Prediction"])

for i in range(0,len(prediction1)):
    writer.writerow([i, prediction1[i]])
result.close()    
submis = pd.read_csv(name)
print submis.shape

(100000, 2)


In [64]:
submis1 = pd.read_csv("./raw/test_sneaky_not_null.csv")
submis2 = pd.read_csv("./raw/test.csv")
print submis1[0:5]
print submis2[0:5]
#k = submis1 - submis2

   Id   Open Date            City  City Group Type  P1  P2  P3  P4  P5 ...   \
0   0  01/22/2011           Niğde       Other   FC   1   4   4   4   1 ...    
1   1  03/18/2011           Konya       Other   IL   3   4   4   4   2 ...    
2   2  10/30/2013          Ankara  Big Cities   FC   3   4   4   4   2 ...    
3   3  05/06/2013         Kocaeli       Other   IL   2   4   4   4   2 ...    
4   4  07/31/2013  Afyonkarahisar       Other   FC   2   4   4   4   1 ...    

   P28  P29  P30  P31  P32  P33  P34  P35  P36  P37  
0    2    3    0    0    0    0    0    0    0    0  
1    1    3    0    0    0    0    0    0    0    0  
2    2    3    0    0    0    0    0    0    0    0  
3    2    3    0    4    0    0    0    0    0    0  
4    5    3    0    0    0    0    0    0    0    0  

[5 rows x 42 columns]
   Id   Open Date            City  City Group Type  P1  P2  P3  P4  P5 ...   \
0   0  01/22/2011           Niğde       Other   FC   1   4   4   4   1 ...    
1   1  03/18/2011   

In [60]:
print k

       Id  Prediction
0       0           0
1       0           0
2       0           0
3       0           0
4       0           0
5       0           0
6       0           0
7       0           0
8       0           0
9       0           0
10      0           0
11      0           0
12      0           0
13      0           0
14      0           0
15      0           0
16      0           0
17      0           0
18      0           0
19      0           0
20      0           0
21      0           0
22      0           0
23      0           0
24      0           0
25      0           0
26      0           0
27      0           0
28      0           0
29      0           0
...    ..         ...
99970   0           0
99971   0           0
99972   0           0
99973   0           0
99974   0           0
99975   0           0
99976   0           0
99977   0           0
99978   0           0
99979   0           0
99980   0           0
99981   0           0
99982   0           0
99983   0 